# Wireless Infrared Bootloader for the RP2040
#### [V. Hunter Adams](https://vanhunteradams.com)

***

## What is a bootloader?

On a microcontroller, a *bootloader* is a program that is responsible for loading and running other programs. The bootloader is how the microcontroller knows how to receive, load, and start executing the programs that we write for it.

Given that we are able to program the microcontrollers that we buy, perhaps it stands to reason that part of the bootloader is "baked in" to ROM, making it difficult or impossible to change. If the microcontroller didn't come with the bootloader already loaded into memory, it wouldn't know how to receive or execute any new programs. And if our programs were to overwrite the bootloader, we would brick our microcontroller by making it impossible to reprogram.

That said, it's often the case that some of the later "stages" of the bootloader do live in programmable memory. The RP2040, for example, uses to two-stage bootloader. The first stage lives in ROM, and the second lives at the beginning of each program. We'll discuss each, and then we'll add our *own* third-stage bootloader. This custom third-stage bootloader will allow for reprogramming via a wireless infrared link, and will allow for RP2040's to send their *own* application code to neighboring RP2040's. This allows for a firmware update to spread, virus or computer-worm-like, through a collection of microcontrollers.

But let's start with an overview of the existing RP2040 bootloader and boot sequence.

## RP2040 boot sequence

The RP2040 datasheet separates the boot sequence into the "hardware-controlled" section which happens *before* the processors begin executing the bootrom, and the "processor-controlled" section during which processor cores 0 and 1 begin to execute the bootrom. The second stage of the bootloader (which lives at the beginning of the user's program) then runs *after* the bootrom.

Let us consider each of these steps in turn, and then we'll add another step to this process.

#### Hardware-controlled boot sequence (baked in)

The hardware-controlled boot sequence is responsible for safely turning on the minimum number of peripherals that are required for the processors to begin executing code. Starting from a totally powered-off RP2040, the sequence is as follows:

1. Power is applied to the chip, and the `RUN` pin is high. The chip will be *held in reset* for as long as `RUN` is not high.
2. The RP2040's on-chip voltage regulator automatically powers on as soon as an input supply is available. The system waits until the digital core supply (DVDD) which comes from this voltage regulator is stable.
3. The Power-On State Machine starts. Various hardware blocks and peripherals must be powered on in a particular order for everything to startup properly, since some require that others are already running (for example, we don't want to start the clock generators until we have a stable clock source). The power-on state machine brings a series of peripherals out of reset in a safe order. In particular:
> - The ring oscillator is started. Once it's stable . . .
> - The crystal oscillator is reset (though not yet started, this occurs in `runtime_init`)
> - The `clk_ref` and `clk_sys` clock generators are brought out of reset. `clk_ref` runs using the ring oscillator with no divider, and `clk_sys` runs from `clk_ref`. In other words, we start clocking the system off the ring oscillator.
> - The reset controller resets all non-boot peripherals
> - The system checks some registers to confirm the boot state of the chip, and whether there's an invalid program in flash
> - The system brings XIP, ROM/SRAM, and the bus fabric out of reset
> - The system brings the processors out of reset, which begin executing bootcode from ROM

<figure>
    <img align="center" width="500" height="500" src="power_on.png" alt='missing' />
    <center><figcaption>Power-on state machine (from RP2040 datasheet)</figcaption></center>
</figure>

This is a *fully automated process*. This whole sequence comes baked into the RP2040 and requires no input from the user to work. There exist some registers that user software can use to override and check the status of the power-on state machine, or the user can configure a watchdog to restart the power-on state machine, but user code does not include this process nor overwrite it when we reprogram the chip.


#### Processor-controlled boot sequence (baked in)

The processor-controlled boot sequence runs from ROM. This sequence too is "baked in" to the RP2040 when we purchase it. This sequence is summarized in the flowchart below. This sequence begins on line 225 of [this file from the bootrom](https://github.com/raspberrypi/pico-bootrom/blob/master/bootrom/bootrom_rt0.S) and it concludes in `main()` of [this file](https://github.com/raspberrypi/pico-bootrom/blob/master/bootrom/bootrom_main.c). In brief, the RP2040 reads the `SIO_CPUID1` register to confirm that it is core 0. If it's not core 0, it stalls and waits to be started by core 0. If it *is* core 0, then it proceeds to the next step of the process. In the event that the rescue flag is set, the core halts to wait for instruction from a debugger. Otherwise, the core checks the watchdog scratch registers for some magic numbers to see whether it should boot to some code loaded into SRAM. If it finds those magic numbers, it vectors to that code. If not, it branches to `main()` in `bootrom_main.c`.

In the event that the user is *not* holding down the `BOOTSEL` button, the RP2040 will attempt a flash boot. To do so, it initializes the SSI channel over which it communicates with external flash memory and associates a particular set of GPIO's with that channel. Then, it starts looking for the start of a valid application! It does so by repeatedly loading the first 256 bytes from flash memory to SRAM using all possible combinations of SSI clock phase and polarity. For each attempt, it computes a checksum to see if it read a valid boot2 second stage in flash memory (it tries all combinations because it doesn't know precisely which external chip it might be communicating with, and different ones may use different modes).

In the event that it finds a valid boot2, it branches to the boot2 entry point! Otherwise, it enters the USB device mode to wait for a new program from the user over the USB interface.


<figure>
    <img align="center" width="500" height="500" src="processor_controlled.png" alt='missing' />
    <center><figcaption>Processor-controlled boot sequence (from RP2040 datasheet)</figcaption></center>
</figure>

#### Boot stage 2 (lives in first 256 bytes of flash)

The RP2040 stores its program code in *external* flash memory, and it is compatible with a number of different external flash chips. All compatible chips use a QSPI interface, which allows for the bootrom to copy the first 256 bytes of flash memory (where the boot2 code lives) into SRAM. This bootcode is executed from SRAM because the rest of flash remains inaccessible to the XIP address window until some further (flash-chip-specific) configurations are performed. These configurations are the responsibility of the boot2 code.

If you look in the SDK, you'll find a [whole library](https://github.com/raspberrypi/pico-sdk/tree/master/src/rp2_common/boot_stage2) of different *versions* of boot2. Each of these versions corresponds to a different flash memory chip, and each is *exactly* 256 bytes long (padded if need be). At compile-time, the compiler checks a flag to determine which flash memory chip is being used and loads the appropriate boot2 code. *This is why boot2 isn't baked into ROM.* We need for the programmer to be able to select the right configurations for their particular flash chip, which wouldn't be possible if the code came baked in. You'll find these flags in the [board files](https://github.com/raspberrypi/pico-sdk/tree/master/src/boards/include/boards) for RP2040-based boards. For instance, the [pico.h](https://github.com/raspberrypi/pico-sdk/blob/master/src/boards/include/boards/pico.h) file specifies `PICO_BOOT_STAGE2_CHOOSE_W25Q080 1`. This is a message to the compiler to please select the [particlar boot2 program](https://github.com/raspberrypi/pico-sdk/blob/master/src/rp2_common/boot_stage2/boot2_w25q080.S) associated with the W25Q080 flash chip.

## Creating one's own bootloader for the RP2040

## Dissecting the infrared bootloader

